In [0]:
###############################################################################
# SETUP - EXECUTE THIS FIRST                                                  #
###############################################################################
# 1. Go into "Runtime" -> "Change runtime type" and Select "GPU" for hardward accelerator
# 2. Click the "Connect" button, at the right to start the instance.
# This will get the dataset into this instance
!wget https://github.com/nknewlands/DeepLearningShortCourse/raw/master/Day1/Pineapple_and_Orange/new_fruits.zip
!unzip new_fruits.zip

--2019-08-12 13:52:00--  https://github.com/nknewlands/DeepLearningShortCourse/raw/master/Day1/Pineapple_and_Orange/new_fruits.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/nknewlands/DeepLearningShortCourse/master/Day1/Pineapple_and_Orange/new_fruits.zip [following]
--2019-08-12 13:52:05--  https://raw.githubusercontent.com/nknewlands/DeepLearningShortCourse/master/Day1/Pineapple_and_Orange/new_fruits.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12125632 (12M) [application/zip]
Saving to: ‘new_fruits.zip’

new_fruits.zip      100%[===================>]  11.56M  --.-KB/s    in 0.09s 

In [0]:
#!/usr/bin/python
#
# Fruits part 2. Using transfer learning with MobileNet
# Inpired by the code from Francois, Chollet. 
#                     "Deep learning with Python." Manning (2018) 362 pages.
# And 
# https://www.pyimagesearch.com/2019/06/24/change-input-shape-dimensions-for-fine-tuning-with-keras/
###############################################################################
# Global import                                                               #
###############################################################################
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.applications import nasnet
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import RMSprop
from keras.callbacks import *
from keras import backend as K

###############################################################################
# HELPER FUNCTIONS                                                            #
###############################################################################
def number_of_files(dirname):
    cpt = sum([len(files) for r, d, files in os.walk(dirname)])
    return cpt

################################################################################ 
# DEFINITION OF INPUT DATA                                                     #
################################################################################
test_data_dir='new_fruits'
epochs = 50    # Number of iteration over the dataset
batch_size = 5 # Number of images processed at the same time
nb_test_samples=number_of_files(test_data_dir)
img_width, img_height = 224, 224 # Needed dimensions for most model

################################################################################ 
# ResNet, MobileNet or other                                                   #
################################################################################
#See available models at https://keras.io/applications/#documentation-for-individual-models
model_nasnet = nasnet.NASNetMobile(weights="imagenet",input_shape=(img_width, img_height, 3))

test_generator = ImageDataGenerator(preprocessing_function = nasnet.preprocess_input).flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
	shuffle=False,
	)

predicted=model_nasnet.predict_generator(test_generator,steps = nb_test_samples // batch_size)

y_pred=nasnet.decode_predictions(predicted, top=1)
results=pd.DataFrame(np.concatenate(y_pred), columns=['imagenet_id','predict_class','percent']) 
results["filename"]=test_generator.filenames
print(results)
results.to_csv("predicted.csv", sep=",")


Using TensorFlow backend.
W0812 13:52:13.232652 139643386144640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0812 13:52:13.272559 139643386144640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 13:52:13.280072 139643386144640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0812 13:52:13.327768 139643386144640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session inste

24231936/24227760 [==============================] - 1s 0us/step
Found 30 images belonging to 2 classes.
40960/35363 [==================================] - 0s 0us/step
   imagenet_id predict_class     percent                       filename
0    n12620546           hip   0.6390811             apples/apples1.jpg
1    n12620546           hip  0.87717247             apples/apples2.jpg
2    n12620546           hip   0.1899739             apples/apples3.jpg
3    n07754684     jackfruit   0.8219086             apples/apples4.jpg
4    n12620546           hip  0.92733884             apples/apples5.jpg
5    n07749582         lemon  0.23427123             apples/apples6.jpg
6    n12620546           hip  0.79833865             apples/apples7.jpg
7    n07749582         lemon   0.7393232             apples/apples8.jpg
8    n03991062           pot  0.33480656    oranges/Orange-valencia.jpg
9    n07747607        orange  0.55466527  oranges/orange-val-copy-1.jpg
10   n09468604        valley  0.32215124

In [0]:
################################################################################ 
# UPLOAD THE PREDICTION TO GOOGLE.DRIVE                                        #
################################################################################
# Display current instance files
!ls -alrth
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')
# Copy the predicted.cvs to your google drive
!cp predicted.csv /content/drive/'My Drive'/


################################################################################
# TO DO                                                                        #
################################################################################
# 1. Train for the apples
# 2. Test other models 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: target 'Drive/' is not a directory
